<a href="https://colab.research.google.com/github/naldo14/tesis-/blob/main/casewest_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import h5py #librerua para crear archivos , h5py
import sys
import numpy as np
import math
from scipy import signal    #aqui estoy usando la libreria signal
import scipy.io
import os
import pandas as pd
import pywt

In [2]:
#hacer el mounted de mi cuenta drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#celda para crear la lista con los paths de mi data
paths=[] #creo la lista de pahts
names=[] #creo la lista de nombres
output_file=[]
paths.append('/content/drive/MyDrive/tesis USB /case west reserve university')
names.append('casewest')
output_file.append('/content/drive/MyDrive/tesis USB /case west reserve university/data')

In [ ]:
# #hago el unar del archivo
# !pip install rarfile

# from google.colab import drive
# import rarfile

# # Path to the .zip file in Google Drive
# zip_file_path = paths[0]+'/'+names[0]+".rar"

# with rarfile.RarFile(zip_file_path, 'r') as rf:
#     rf.extractall(output_file[0])

In [ ]:
# #creo que los h5py file vacios
# import os
# import h5py

# # List of directory paths
# directory_paths = [
#     "/content/drive/MyDrive/tesis USB /case west reserve university/data/inner",
#     "/content/drive/MyDrive/tesis USB /case west reserve university/data/outter",
#     "/content/drive/MyDrive/tesis USB /case west reserve university/data/outter1",
#     "/content/drive/MyDrive/tesis USB /case west reserve university/data/normal"
# ]

# # Loop through each directory path
# for directory_path in directory_paths:
#     # Get the directory name (e.g., "inner," "outter," etc.)
#     directory_name = os.path.basename(directory_path)

#     # Create an h5py file for the directory
#     h5py_file = h5py.File(f"{directory_name}.h5", "w")

#     # List .mat files in the directory
#     mat_files = [f for f in os.listdir(directory_path) if f.endswith(".mat")]

#     # Add each .mat file to the h5py file as an empty dataset
#     for mat_file in mat_files:
#         # Create an empty dataset
#         dataset = h5py_file.create_dataset(mat_file, shape=(0,), dtype='f4')

#     # Close the h5py file
#     h5py_file.close()

# Now you have separate h5py files for each directory with empty datasets

In [ ]:

# ### function to storaged all the .mat file into the h5py's file
# def save_mat_to_h5py(mat_files_directories, h5py_file_names):
#     if len(mat_files_directories) != len(h5py_file_names):
#         print("Error: The number of directories and h5py file names should match.")
#         return

#     for mat_files_directory, h5py_file_name in zip(mat_files_directories, h5py_file_names):
#         # Open the h5py file
#         with h5py.File(h5py_file_name, 'w') as h5file:
#             # Get the list of .mat files in the directory
#             mat_files = [f for f in os.listdir(mat_files_directory) if f.endswith(".mat")]

#             # Add each .mat file content to the h5py file
#             for mat_file in mat_files:
#                 # Load the ..mat file
#                 mat_data = scipy.io.loadmat(os.path.join(mat_files_directory, mat_file))

#                 # Create datasets for each variable in the .mat file
#                 for var_name, var_data in mat_data.items():
#                     h5file.create_dataset(f"{mat_file}/{var_name}", data=var_data)

# mat_files_directories = [
#     '/content/drive/MyDrive/tesis USB /case west reserve university/data/inner',
#     '/content/drive/MyDrive/tesis USB /case west reserve university/data/normal',
#     '/content/drive/MyDrive/tesis USB /case west reserve university/data/outter',
#     '/content/drive/MyDrive/tesis USB /case west reserve university/data/outter1'

# ]
# h5py_file_names = [
#     '/content/drive/MyDrive/tesis USB /case west reserve university/inner.h5',
#     '/content/drive/MyDrive/tesis USB /case west reserve university/normal.h5',
#     '/content/drive/MyDrive/tesis USB /case west reserve university/outter.h5',
#     '/content/drive/MyDrive/tesis USB /case west reserve university/outter1.h5'
# ]
# save_mat_to_h5py(mat_files_directories, h5py_file_names)

In [10]:
# 'data' now contains a dictionary where the keys are the .mat file names
# and the values are dictionaries containing the arrays for each .mat file

def get_h5py_data(h5py_file_path):
    data = {}

    # Open the h5py file
    with h5py.File(h5py_file_path, 'r') as h5file:
        # List the keys (dataset names) in the h5py file
        keys = list(h5file.keys())

        for key in keys:
            # Access the content of the .mat file (dataset)
            mat_data = h5file[key]

            # Create a dictionary to store the arrays for this .mat file
            arrays = {}

            for subkey, value in mat_data.items():
                if isinstance(value, h5py.Group):
                    # If it's a group (nested data), you can access its keys
                    subkeys = list(value.keys())
                    for subkey in subkeys:
                        # Access the actual data (arrays) and store in the dictionary
                        subkey_data = value[subkey][()]
                        arrays[subkey] = subkey_data
                else:
                    # If it's a dataset (e.g., a variable), you can access its value
                    # and store it in the dictionary using the original key
                    variable_data = value[()]
                    arrays[subkey] = variable_data

            # Store the arrays for this .mat file in the data dictionary
            data[key] = arrays

    return data


In [11]:

def band_maker(coeffs, fs, title):
    p = len(coeffs)- 1
    low_frequencies = [None] * len(coeffs)
    high_frequencies = [None] * len(coeffs)

    # low_frequencies[0] = 0
    # high_frequencies[0] = fs / (2 ** (p) * 2)
    for i in range(0, len(coeffs)):
         high_frequencies[i] = fs / (2 ** (p+-i + 1))
    low_frequencies[0]=0
    low_frequencies[1:len(low_frequencies)]= high_frequencies[:len(low_frequencies) - 1]



    return  high_frequencies,low_frequencies

In [4]:
def frequencies_maker(N,fs,d,D,theta):
  theta=(theta/180)*np.pi
  FTF=(fs/2)*(1-d/D*np.cos(theta))
  BPFI=(N*fs/2)*(1+d/D*np.cos(theta))
  BPFO=(N*fs/2)*(1-d/D*np.cos(theta))
  BSF=(D*fs/d)*(1-(d/D)**2*np.cos(theta))
  return(FTF,BPFI,BPFO,BSF)


In [5]:
FTF,BPFI,BPFO,BSF=frequencies_maker(9,50.17,7.90,38.5,0) #estos datos son para el daata set coreano
data = {
    'Name': ['Fundamental train frequency (FTF)', 'Ball pass frequency inner (BPFI)', 'Ball pass frequency outer (BPFO)','Ball spin frequency (BSF)'],
    'Values (HZ)': [FTF, BPFI, BPFO,BSF]
}

# Create a dataframe from the dictionary
df = pd.DataFrame(data)

# Print the dataframe
print(df)

                                Name  Values (HZ)
0  Fundamental train frequency (FTF)    19.937688
1   Ball pass frequency inner (BPFI)   272.090805
2   Ball pass frequency outer (BPFO)   179.439195
3          Ball spin frequency (BSF)   234.204744


In [6]:
def cwt_maker(dic_data, wavelet='db38', level=5):
  w = pywt.Wavelet(wavelet)
  coeffs = pywt.wavedec(dic_data, w, level=level)
  print(level)
  energies_levels = [np.sum(np.abs(coeffs)**2) for coeffs in coeffs]
  array_dwt = energies_levels

  return (np.array(array_dwt))


In [43]:
#aqui solo normalizo los niveles energeticos

def normalization_from_coeffs(coeffs):

    mean_energy = np.mean(coeffs)
    std_energy = np.std(coeffs)

    # Calculate Z-scores
    z_scores = [(energy - mean_energy) / std_energy for energy in coeffs]

    return z_scores

In [12]:
## almacenamiento de la data inner
file_path = "/content/drive/MyDrive/tesis USB /case west reserve university/outter1.h5"
data_inner = get_h5py_data(file_path)

In [13]:
## almacenamiento de la data outter
file_path = "/content/drive/MyDrive/tesis USB /case west reserve university/outter.h5"
data_outter = get_h5py_data(file_path)

In [14]:
## almacenamiento de la data outter
file_path = "/content/drive/MyDrive/tesis USB /case west reserve university/outter1.h5"
data_outter_1 = get_h5py_data(file_path)

In [15]:
## almacenamiento de la data normal
file_path = "/content/drive/MyDrive/tesis USB /case west reserve university/normal.h5"
data_normal = get_h5py_data(file_path)

In [16]:
## calculo el nivel del banco de filtro:
fs=math.ceil(FTF) #frecuencia natural , se usa la del train frequencie
fe=25.6e3 # frecuencia de sampleo
n = math.ceil(math.log(fe / fs) / math.log(2)) +2
level = n

In [ ]:
pywt.wavelist(family=None, kind='discrete')

['bior1.1',
 'bior1.3',
 'bior1.5',
 'bior2.2',
 'bior2.4',
 'bior2.6',
 'bior2.8',
 'bior3.1',
 'bior3.3',
 'bior3.5',
 'bior3.7',
 'bior3.9',
 'bior4.4',
 'bior5.5',
 'bior6.8',
 'coif1',
 'coif2',
 'coif3',
 'coif4',
 'coif5',
 'coif6',
 'coif7',
 'coif8',
 'coif9',
 'coif10',
 'coif11',
 'coif12',
 'coif13',
 'coif14',
 'coif15',
 'coif16',
 'coif17',
 'db1',
 'db2',
 'db3',
 'db4',
 'db5',
 'db6',
 'db7',
 'db8',
 'db9',
 'db10',
 'db11',
 'db12',
 'db13',
 'db14',
 'db15',
 'db16',
 'db17',
 'db18',
 'db19',
 'db20',
 'db21',
 'db22',
 'db23',
 'db24',
 'db25',
 'db26',
 'db27',
 'db28',
 'db29',
 'db30',
 'db31',
 'db32',
 'db33',
 'db34',
 'db35',
 'db36',
 'db37',
 'db38',
 'dmey',
 'haar',
 'rbio1.1',
 'rbio1.3',
 'rbio1.5',
 'rbio2.2',
 'rbio2.4',
 'rbio2.6',
 'rbio2.8',
 'rbio3.1',
 'rbio3.3',
 'rbio3.5',
 'rbio3.7',
 'rbio3.9',
 'rbio4.4',
 'rbio5.5',
 'rbio6.8',
 'sym2',
 'sym3',
 'sym4',
 'sym5',
 'sym6',
 'sym7',
 'sym8',
 'sym9',
 'sym10',
 'sym11',
 'sym12',
 'sym13',

In [ ]:

# list(data_outter.keys())
data_outter['190.mat'].keys()
# data_outter['135.mat']['X135_DE_time']

dict_keys(['X190RPM', 'X190_DE_time', 'X190_FE_time', '__globals__', '__header__', '__version__'])

In [17]:
reshaped_array_healty = data_normal['100.mat']['X100_DE_time'].reshape((data_normal['100.mat']['X100_DE_time'].shape[0],))
reshaped_array_faulty=data_outter['190.mat']['X190_DE_time'].reshape((data_outter['190.mat']['X190_DE_time'].shape[0]))

In [18]:
coeff_1=cwt_maker(reshaped_array_healty , 'db38', level=level)
coeff_2=cwt_maker(reshaped_array_faulty , 'db38', level=level)

13
13


/usr/local/lib/python3.10/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 13 is too high: all coefficients will experience boundary effects.
  warnings.warn(


In [ ]:
df = pd.DataFrame({'Nivel de energia rodamiento sano ':  coeff_1, 'Nivel de energia rodamiento bajo falla ': coeff_2})
df

,Nivel de energia rodamiento sano,Nivel de energia rodamiento bajo falla
0,174.566402,1292.554268
1,0.256229,12.897994
2,0.134699,7.226738
3,0.073371,3.452078
4,0.087824,1.262455
5,0.157908,2.540037
6,41.353035,46.224431
7,191.671217,727.711137
8,319.882363,788.217601
9,212.629328,275.950575


In [20]:
# Calculate mean and standard deviation

mean_energy = np.mean(coeff_2)
std_energy = np.std(coeff_2)
# Calculate Z-scores
z_scores = [(energy - mean_energy) / std_energy for energy in coeff_2]

# Normalize the energies_faulty array
normalized_energies_faulty = z_scores

In [21]:

mean_energy = np.mean(coeff_1)
std_energy = np.std(coeff_1)

# Calculate Z-scores
z_scores = [(energy - mean_energy) / std_energy for energy in coeff_1]

# Normalize the energies_healty array
normalized_energies_healty = z_scores


In [41]:
coeff_1

array([1.74566402e+02, 2.56229295e-01, 1.34699222e-01, 7.33712765e-02,
       8.78242847e-02, 1.57907642e-01, 4.13530350e+01, 1.91671217e+02,
       3.19882363e+02, 2.12629328e+02, 2.02346871e+02, 6.00483111e+02,
       4.74347942e+02, 3.88858148e+00])

In [22]:
df = pd.DataFrame({'Nivel de energia rodamiento sano normalizado ':  normalized_energies_healty, 'Nivel de energia rodamiento bajo falla normalziado': normalized_energies_faulty})
df

,Nivel de energia rodamiento sano normalizado,Nivel de energia rodamiento bajo falla normalziado
0,0.084747,0.265524
1,-0.846621,-0.511488
2,-0.847271,-0.514932
3,-0.847598,-0.517224
4,-0.847521,-0.518553
5,-0.847147,-0.517777
6,-0.627034,-0.491252
7,0.176141,-0.077451
8,0.861194,-0.040711
9,0.288123,-0.351761


In [46]:
## funcion para calcular los coeficientes normalizados de todo un set
def get_data(file_path, wavelet, level):
    # Get the data from the H5PY file
    data = get_h5py_data(file_path)

    # Get the order of experiments (sorted keys)
    experiment_order = sorted(data.keys())

    # Create an empty list to store the coefficients
    coefficients_list = []

    # Process each experiment and calculate coefficients
    for experiment in experiment_order:
        # Reshape the data for the current experiment
        reshaped_data = data[experiment][list(data[experiment].keys())[1]].reshape(-1)

        # Calculate CWT coefficients for the experiment
        coefficients = cwt_maker(reshaped_data, wavelet, level)
        coefficientes_normalized=normalization_from_coeffs(coefficients)
        # Append the coefficients to the list
        coefficients_list.append(coefficientes_normalized)

    return coefficients_list

In [50]:
len(data_outter.keys())

17

In [52]:
 experiment_order = sorted(data_outter.keys())
 experiment_order

['135.mat',
 '136.mat',
 '137.mat',
 '138.mat',
 '190.mat',
 '191.mat',
 '192.mat',
 '201.mat',
 '202.mat',
 '204.mat',
 '226.mat',
 '227.mat',
 '228.mat',
 '238.mat',
 '239.mat',
 '240.mat',
 '241.mat']

In [37]:
 data_outter[experiment_order[0]][list(data_outter[experiment_order[0]].keys())[1]]

array([[ 0.89351067],
       [ 0.58844   ],
       [ 0.15441333],
       ...,
       [-0.07094667],
       [-0.117688  ],
       [-0.14439733]])

In [35]:
data_outter[experiment_order[0]]['X135_DE_time']

array([[ 0.89351067],
       [ 0.58844   ],
       [ 0.15441333],
       ...,
       [-0.07094667],
       [-0.117688  ],
       [-0.14439733]])

In [48]:
file_path= "/content/drive/MyDrive/tesis USB /case west reserve university/outter.h5"
normalized_outter= get_data(file_path, 'db38', level)

/usr/local/lib/python3.10/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 13 is too high: all coefficients will experience boundary effects.
  warnings.warn(


13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13


In [54]:
normalized_outter[4]==df['Nivel de energia rodamiento bajo falla normalziado']

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
Name: Nivel de energia rodamiento bajo falla normalziado, dtype: bool